# Bilibili Scraping  - Danmaku 弹幕爬取

## Method with history danmaku

In [2]:
# import library
import requests
from bs4 import BeautifulSoup as bs
import json
import pandas as pd
import datetime
import time 
import xml.etree.ElementTree as etree
import re
from selenium import webdriver

In [ ]:
# https://api.bilibili.com/x/v1/dm/list.so?oid=45508213

In [3]:
# convert aid to cid 
aid_list_df = pd.read_csv('共青团info_cleaned.csv')
def get_cid(aid): 
    # api of video's cid 
    url_cid = 'https://www.bilibili.com/widget/getPageList?aid='+ str(aid)
    # page content: [{"page":1,"pagename":"\u3010\u9999\u6e2f\u771f\u76f8\u5927\u8d77\u5e95\u3011\u66b4\u529b\u4e71\u6e2f\u5b9e\u5f55","cid":130684654}]
    response_cid = requests.get(url_cid)
    json_cid = json.loads(response_cid.text)
    cid = json_cid[0]['cid']
    return str(cid)
aid_list_df['cid'] = aid_list_df["aid"].apply(get_cid)
aid_list_df

,Unnamed: 0,aid,bvid,title,typeid,section,sub-section,date,time,length,play,comment,danmaku,favorite,coin,share,like,typename,his_rank,cid
0,0,76394931,BV1cJ411274r,【香港真相大起底】暴力乱港实录,203.0,资讯,热点,2019-11-20 12:48:34,1574254114,09:27,10061835,35092.0,272511,358453,517252,375956,623387,热点-资讯,1,130684654
1,1,76410258,BV19J411174N,今天新闻联播，12条连发批美国国会参议院通过“香港人权与民主法案”,203.0,资讯,热点,2019-11-20 13:53:30,1574258010,13:58,802925,5397.0,8113,8190,12277,8396,57048,热点-资讯,0,130701291
2,2,76467274,BV13J41127xi,【中国有故事 第10集】一条铁路永流传,124.0,科技,社科人文,2019-11-21 06:04:03,1574316243,03:59,70372,17.0,0,410,1678,227,10486,社科人文-科技,0,130800713
3,3,76485537,BV1bJ41127Wb,警方公布英驻香港总领馆雇员嫖娼审讯视频：不存在刑讯逼供,203.0,资讯,热点,2019-11-21 09:13:38,1574327618,02:11,786010,2410.0,1971,2227,4604,3363,33643,热点-资讯,0,130833155
4,4,76524390,BV1zJ411271R,中国人唱给美国的魔性RAP,206.0,资讯,综合,2019-11-21 14:48:08,1574347688,02:05,5084854,10398.0,12647,276440,479748,73857,578104,综合-资讯,2,130891265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,350,925637499,BV1ET4y1u7AQ,可可西里：4479米的牺牲与青春,21.0,生活,日常,2020-05-18 08:02:56,1589788976,09:26,69228,442.0,574,765,2387,248,19383,日常-生活,0,192416711
351,351,498199128,BV16K411W7pJ,【另一个香港】第一集 香港之乱,203.0,资讯,热点,2020-05-18 08:24:17,1589790257,34:19,494472,2222.0,20065,15262,15787,7657,47224,热点-资讯,96,192442683
352,352,925675319,BV1bT4y1u7yK,【另一个香港】第二集 风从何来,203.0,资讯,热点,2020-05-18 09:28:21,1589794101,39:19,224343,1341.0,11675,5113,7694,2348,27512,热点-资讯,0,192470876
353,353,285685867,BV1Hf4y1U7Y9,香港历史教科书，能教出怎样的学生？,96.0,科技,星海,2020-05-18 11:05:06,1589799906,10:03,339853,2918.0,3668,3239,3484,1321,35652,星海-科技,0,192482699


In [ ]:
aid_list_df.to_csv('共青团info_cleaned_cid.csv') # write out the df with cid 
aid_list_df = pd.read_csv('共青团info_cleaned_cid.csv') # read df with cid 

### Danmaku scraping and reformatting

In [ ]:
def get_danmaku(aid,cid,date):

    #cid = "130684654"


    url = 'https://api.bilibili.com/x/v2/dm/history?type=1&oid='+str(cid)+'&date=%s'%date
    driver.get(url)
    soup = bs(driver.find_element_by_tag_name('i').get_attribute('innerHTML'), 'lxml')
    soup = str(soup)

    format_doc = re.compile('<d p="(.*?)">(.*?)</d>')
        # danmu = [(('54.01300,1,25,16777215,1531313705,0,f1d4f711,2187083291033602','我狂铁胜率百分之85')),()]
        # () = tuple
    danmu = format_doc.findall(soup)


    def reformat(tuples):
        # info = '54.01300,1,25,16777215,1531313705,0,f1d4f711,2187083291033602' - str
        info = tuples[0]
        # words = '我狂铁胜率百分之85'
        words = tuples[1]
        # info = [54.01300,1,25,16777215,1531313705,0,f1d4f711,2187083291033602] - list
        # info = [弹幕出现的时间,弹幕的模式]
        info = info.split(',')
        # timestamp, readable datatime 
        #return [[['1531313705', '2018-07-11 12:55:05', '我狂铁胜率百分之85']],[],[]]
        return [info[4],str(pd.to_datetime(info[4],unit='s')),words]

    map_f = map(reformat, danmu)
    data=list(map_f)


    df_new = pd.DataFrame(data,columns=['dm_timestamp', 'dm_date', 'dm_content'])
    df_new['aid'] = str(aid)
    df_new['cid'] = str(cid)
    df_new = df_new[['aid','cid','dm_timestamp', 'dm_date', 'dm_content']]

    # remove hours in date, 2019-11-25 15:59:17 ==> 2019-11-25 
    def get_date(input_dates):
        return str(datetime.datetime.strptime(input_dates, '%Y-%m-%d %H:%M:%S').date())
    df_new["dm_date"]=df_new["dm_date"].apply(get_date)
    return df_new

因为b站有反爬虫机制，如果短时间内刷新太多，会将b站账号和ip禁封一段时间。因此我将所有需要爬取弹幕的视频分成几段分批爬取。   
Because Bilibili has an anti-scraping mechanism, if the pages refreshes too many times in a short time, it will block the account and ip address for about 20min-2hours. So I splits all the videos into several parts.

In [143]:
t1 = "18/01/2020 00:00:00"
t2 = "08/04/2020 00:00:00"
begin_date = time.mktime(datetime.datetime.strptime(t1, "%d/%m/%Y %H:%M:%S").timetuple())
end_date = time.mktime(datetime.datetime.strptime(t2, "%d/%m/%Y %H:%M:%S").timetuple())

#  videos released during the Wuhan lockdown
#  aid_list_df2 = aid_list_df2[(aid_list_df2['time']>= begin_date) & (aid_list_df2['time']<=end_date)]

#  videos released before the Wuhan lockdown
#  aid_list_df3 = aid_list_df3[(aid_list_df3['time']<=begin_date)]
#  videos released after the Wuhan lockdown
aid_list_df3 = aid_list_df3[(aid_list_df3['time']>= end_date)]
aid_list_df3.head(1)

,aid,bvid,title,typeid,section,sub-section,date,time,length,play,comment,danmaku,favorite,coin,share,like,typename,his_rank,cid
252,752650393,BV12k4y197yY,《手足》英雄的城，英雄来守护,203.0,资讯,热点,2020-04-08 08:52:47,1586335967,04:49,41515,262.0,209,718,1950,258,16044,热点-资讯,0,175492753


In [144]:
pd.set_option('display.max_rows', 500)
# remove the video without danmaku
aid_list_df3=aid_list_df3.loc[aid_list_df3["danmaku"]>10,:]
aid_list_df3

,aid,bvid,title,typeid,section,sub-section,date,time,length,play,comment,danmaku,favorite,coin,share,like,typename,his_rank,cid
252,752650393,BV12k4y197yY,《手足》英雄的城，英雄来守护,203.0,资讯,热点,2020-04-08 08:52:47,1586335967,04:49,41515,262.0,209,718,1950,258,16044,热点-资讯,0,175492753
253,710228623,BV1BQ4y1K73Q,热干面回来了，我们陪你再过个年！,203.0,资讯,热点,2020-04-08 10:29:23,1586341763,00:22,190424,2205.0,987,2593,2183,916,41281,热点-资讯,0,175615624
254,795153782,BV1aC4y1s7bf,【向上的力量 第一期】中国人的“星辰大海”是什么？杨利伟用十分钟告诉你！,39.0,科技,演讲· 公开课,2020-04-08 11:59:21,1586347161,10:21,129398,852.0,1609,2530,5159,781,29509,演讲· 公开课-科技,0,176323877
256,370204613,BV1TZ4y1x72r,“我叫志愿者！”,203.0,资讯,热点,2020-04-09 03:52:06,1586404326,05:06,62451,332.0,112,568,1667,440,18831,热点-资讯,0,175890020
257,242650112,BV1Ze411x7Si,世卫组织回应美国：反对政治化新冠肺炎,203.0,资讯,热点,2020-04-09 04:51:46,1586407906,00:48,179122,825.0,119,439,759,244,26187,热点-资讯,0,175920351
258,327719531,BV1KA411b7Dm,世卫总干事：我就直说了，骂我的来自台湾岛内,203.0,资讯,热点,2020-04-09 10:04:25,1586426665,01:29,1872219,5499.0,8186,8423,5207,5131,116445,热点-资讯,90,176048095
259,242663352,BV1we411476B,结尾高能！美国劳工部统计的每周首次申请失业救济人数,206.0,资讯,综合,2020-04-09 14:10:54,1586441454,01:27,521162,7524.0,6677,5820,5942,11818,44889,综合-资讯,0,176167125
260,242724740,BV1ne41147dX,【中华谣】融入琴棋书画诗词歌赋...超萌国风单曲来了！,28.0,音乐,原创音乐,2020-04-10 06:00:49,1586498449,03:46,156576,1069.0,1180,8414,9055,1310,30217,原创音乐-音乐,0,176010336
261,455175815,BV1A5411t76w,【向上的力量 第二期】“中国电影不仅要走出去，更要走进来！”,39.0,科技,演讲· 公开课,2020-04-10 09:29:38,1586510978,10:20,97943,1369.0,202,354,1137,126,18084,演讲· 公开课-科技,0,176482042
262,882726253,BV1rK4y1r7Ew,【青年讲师谈】第二讲：民国“疫”事,39.0,科技,演讲· 公开课,2020-04-10 10:51:46,1586515906,10:55,49441,287.0,337,422,1416,234,14390,演讲· 公开课-科技,0,176513150


In [28]:
#pd.to_datetime("1586616147",unit='s')
#pd.datetime.strptime("2020-02-01", '%Y-%m-%d') - pd.datetime.strptime("2020-04-20", '%Y-%m-%d')

Timestamp('2020-04-11 14:42:27')

In [182]:
# in order to obtain the history danmaku, we need to log in to bilibili
url = 'https://passport.bilibili.com/login'
driver = webdriver.Chrome("/Applications/chromedriver")
driver.get(url)
input('请在浏览器登录账号后回车')

In [229]:
# obtain danmaku of videos (during the lockdown)
# create an empty df to store danmaku 
df = pd.DataFrame(columns=["aid","cid",'dm_timestamp', 'dm_date', 'dm_content'])

# go through each video
for i in list(aid_list_df3.cid)[75:]:

    aid = str(aid_list_df3.loc[aid_list_df3.cid==i,"aid"].values[0])
    
    # remove the hour from dates 
    def get_date(input_dates):
        return str(datetime.datetime.strptime(input_dates, '%Y-%m-%d %H:%M:%S').date())
    dates = get_date(aid_list_df3.loc[aid_list_df3.cid==i,"date"].values[0])
    
    # get history dates of videos
    org = pd.datetime.strptime(dates, '%Y-%m-%d')
    # start date - the day that video released
    start = pd.datetime.strptime(dates, '%Y-%m-%d')
    # end date - 2020-05-20
    end  =  pd.datetime.strptime("2020-05-20", '%Y-%m-%d')
    
    # print out the video name for tracking the progress.
    print (str(aid_list_df3.loc[aid_list_df3.cid==i,"title"].values[0]),
           str(aid_list_df3.loc[aid_list_df3.cid==i,"aid"].values[0]))
    
    # go through all the history dates and scrape the danmaku 
    while start <= end:
      # https://api.bilibili.com/x/v2/dm/history?type=1&oid=143873534&date=2020-05-20
        date = pd.datetime.strftime(start, '%Y-%m-%d')

        # obtain danmaku of specific date
        df_new = get_danmaku(aid,i,date) 
        # merge new data to the df 
        df = pd.concat([df, df_new], axis=0, sort=False)
        # remove deplicated rows, 
        # if the number of danmaku less than the limit of danmaku pre pages, the danmaku that post on the pervious day will be included 
        df.drop_duplicates(inplace=True)
        
        # go to next dates 
        # since the most danmaku is posted within 5 day of video posted. 
        if (start-org).days<6:
            start =  start + datetime.timedelta(days=1)
        else:
            start = start + datetime.timedelta(days=30)
    
    df_new = get_danmaku(aid,i,"2020-05-20")
    df = pd.concat([df, df_new], axis=0, sort=False)
    df.drop_duplicates(inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  del sys.path[0]
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  app.launch_new_instance()
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.


【云开日出 重拾烟火】B站美食纪录片大混剪，云开日出，再陪武汉过个年 498062081
【我会找到你】搜救犬宣传片2020【献给无言的战士】 285668964
新闻联播：中国人不是被吓大的！ 840672960
5年贪污300万，竟是为了买玩具手办…… 883207768
【罗伯，一路好走】罗伯去世半周年忌日，向佐等市民前来吊唁 710661477
中国黑客偷美国疫苗？外交部：我们才担心被偷 668211062
向救死扶伤者，致敬！ 840682726
亚丁湾开帆船偶遇人民海军是一种怎样的体验 838192051
起底美国去年被关的生物研究基地 838140541
假如闭上双眼，你敢在黑暗中奔跑吗？ 328146940


In [230]:
df3 = df
df3 

,aid,cid,dm_timestamp,dm_date,dm_content
0,84104587,143873534,1579449544,2020-01-19,你需要你把qbu11
1,84104587,143873534,1579449430,2020-01-19,？？？
2,84104587,143873534,1579449415,2020-01-19,蹲空投阴人
3,84104587,143873534,1579449404,2020-01-19,先封烟！先封烟！
4,84104587,143873534,1579449220,2020-01-19,封烟强拉
...,...,...,...,...,...
36,328146940,192127980,1589912222,2020-05-19,你们真厉害，只是你们不知道
37,328146940,192127980,1589908501,2020-05-19,你们真厉害
38,328146940,192127980,1589908293,2020-05-19,我闭着眼走路会走歪，
39,328146940,192127980,1589905946,2020-05-19,国足努力了？那为什么踢不过其他国家的？


In [231]:
#df.drop_duplicates(inplace=True)
df.shape

(583989, 5)

In [228]:
# pd.set_option('display.max_rows', 500)
# aid_list_df3.loc[:,["length","cid","title","danmaku","date"]].iloc[92:,]

,length,cid,title,danmaku,date
349,00:38,192322499,“抓小偷！武警大院门口！”哨兵：好嘞,796,2020-05-18 00:44:09
350,09:26,192416711,可可西里：4479米的牺牲与青春,574,2020-05-18 08:02:56
351,34:19,192442683,【另一个香港】第一集 香港之乱,20065,2020-05-18 08:24:17
352,39:19,192470876,【另一个香港】第二集 风从何来,11675,2020-05-18 09:28:21
353,10:03,192482699,香港历史教科书，能教出怎样的学生？,3668,2020-05-18 11:05:06
354,02:09,192597418,【中美对比】中国战“疫”时，美国在做什么？,884,2020-05-18 14:30:57


In [232]:
#aid_list_df2

,Unnamed: 0,aid,bvid,title,typeid,section,sub-section,date,time,length,play,comment,danmaku,favorite,coin,share,like,typename,his_rank,cid
91,91,84104587,BV1x7411q7R4,【吃鸡技巧】解放军教你甩狙，守株待兔，一石二鸟,96.0,科技,星海,2020-01-19 07:21:07,1579418467,03:37,1224674,3621.0,4899,12750,5230,8584,78566,星海-科技,71,143873534
92,92,84129670,BV1y7411q7T3,【王菲X那英】中国女排《夺冠》片尾曲,203.0,资讯,热点,2020-01-19 10:05:01,1579428301,05:25,133141,1259.0,534,980,1222,858,16710,热点-资讯,0,143915643
93,93,84212847,BV177411v7PM,“习大大，彭麻麻呢？”“没来。快过年了都在家里忙着呢！”,203.0,资讯,热点,2020-01-20 00:40:12,1579480812,00:13,2903691,0.0,0,28283,33750,12029,284675,热点-资讯,6,144049743
94,94,84230689,BV1s7411v755,【喀什地震】铁路值班员不跑，拿起喊停列车,21.0,生活,日常,2020-01-20 04:17:09,1579493829,00:19,565712,1315.0,676,2196,7624,981,61615,日常-生活,0,144078923
95,95,84281160,BV1S7411v7JB,【协和医院】当医生们开始唱歌,31.0,音乐,翻唱,2020-01-20 10:07:44,1579514864,04:28,287449,1701.0,1240,6204,9559,2688,38349,翻唱-音乐,0,144166942
96,96,84382271,BV1H7411i7cm,习近平向全国人民拜年！,203.0,资讯,热点,2020-01-21 04:41:04,1579581664,01:29,387329,0.0,0,3240,8385,1945,66307,热点-资讯,0,144337427
97,97,84400778,BV1L7411i78t,【成长启示录⑥】令人心疼！7岁女童遭虐待，自曝曾被奶奶用老虎钳夹手……,37.0,纪录片,人文历史,2020-01-21 07:50:35,1579593035,36:01,149064,804.0,2267,1097,1788,714,17588,人文历史-纪录片,0,144367118
98,98,84669588,BV1Z7411e7u2,牛津大学国乐宣传演讲：让世界认识中国民族乐器,39.0,科技,演讲· 公开课,2020-01-23 09:59:57,1579773597,14:02,154236,500.0,676,1251,2310,531,21859,演讲· 公开课-科技,0,144808266
99,99,84656702,BV1R7411e7Ff,春节，一起来捏面人吧,203.0,资讯,热点,2020-01-24 02:02:09,1579831329,00:39,72093,528.0,82,110,439,331,13884,热点-资讯,0,144787470
100,100,84802804,BV1d7411a7zJ,【团团有话说】今年春节你在哪儿过年？,21.0,生活,日常,2020-01-24 03:01:05,1579834865,02:19,74155,1216.0,208,153,1118,349,12497,日常-生活,0,145029273


In [235]:
#aid_list_df[['aid',"title","date"]]

,aid,title,date
0,76394931,【香港真相大起底】暴力乱港实录,2019-11-20 12:48:34
1,76410258,今天新闻联播，12条连发批美国国会参议院通过“香港人权与民主法案”,2019-11-20 13:53:30
2,76467274,【中国有故事 第10集】一条铁路永流传,2019-11-21 06:04:03
3,76485537,警方公布英驻香港总领馆雇员嫖娼审讯视频：不存在刑讯逼供,2019-11-21 09:13:38
4,76524390,中国人唱给美国的魔性RAP,2019-11-21 14:48:08
5,76558189,【日本語吹替版】6年前のアニメは今の香港事件を予言しました,2019-11-22 02:42:11
6,76450224,【杜甫 | 剑器行 三无Marblue、祖娅纳惜、玄觞、以冬、竹桑、云泣】一舞剑器动四方！...,2019-11-23 10:00:54
7,76775549,香港警察：你们吃面包，我去吃海底捞！结果他们真去了……,2019-11-23 14:36:26
8,76838810,香港警务处处长邓炳强：香港警队有信心完成止暴制乱任务,2019-11-24 05:09:19
9,76877449,【天府事变】选择【唱给香港】,2019-11-24 13:50:36


In [254]:
# title and date to the df  
df.aid= df.aid.astype(str)
aid_list_df.aid=aid_list_df.aid.astype(str)
df_title=df.merge(aid_list_df[["title","date","aid"]], on='aid', how='left')
df_title

,aid,cid,dm_timestamp,dm_date,dm_content,title,date
0,84104587,143873534,1579449544,2020-01-19,你需要你把qbu11,【吃鸡技巧】解放军教你甩狙，守株待兔，一石二鸟,2020-01-19 07:21:07
1,84104587,143873534,1579449430,2020-01-19,？？？,【吃鸡技巧】解放军教你甩狙，守株待兔，一石二鸟,2020-01-19 07:21:07
2,84104587,143873534,1579449415,2020-01-19,蹲空投阴人,【吃鸡技巧】解放军教你甩狙，守株待兔，一石二鸟,2020-01-19 07:21:07
3,84104587,143873534,1579449404,2020-01-19,先封烟！先封烟！,【吃鸡技巧】解放军教你甩狙，守株待兔，一石二鸟,2020-01-19 07:21:07
4,84104587,143873534,1579449220,2020-01-19,封烟强拉,【吃鸡技巧】解放军教你甩狙，守株待兔，一石二鸟,2020-01-19 07:21:07
...,...,...,...,...,...,...,...
583984,328146940,192127980,1589912222,2020-05-19,你们真厉害，只是你们不知道,假如闭上双眼，你敢在黑暗中奔跑吗？,2020-05-17 10:38:33
583985,328146940,192127980,1589908501,2020-05-19,你们真厉害,假如闭上双眼，你敢在黑暗中奔跑吗？,2020-05-17 10:38:33
583986,328146940,192127980,1589908293,2020-05-19,我闭着眼走路会走歪，,假如闭上双眼，你敢在黑暗中奔跑吗？,2020-05-17 10:38:33
583987,328146940,192127980,1589905946,2020-05-19,国足努力了？那为什么踢不过其他国家的？,假如闭上双眼，你敢在黑暗中奔跑吗？,2020-05-17 10:38:33


In [256]:
# reformate dates
def get_date(input_dates):
    return str(datetime.datetime.strptime(input_dates, '%Y-%m-%d %H:%M:%S').date())
df_title["date"]=df_title["date"].apply(get_date)

In [257]:
df_title

,aid,cid,dm_timestamp,dm_date,dm_content,title,date
0,84104587,143873534,1579449544,2020-01-19,你需要你把qbu11,【吃鸡技巧】解放军教你甩狙，守株待兔，一石二鸟,2020-01-19
1,84104587,143873534,1579449430,2020-01-19,？？？,【吃鸡技巧】解放军教你甩狙，守株待兔，一石二鸟,2020-01-19
2,84104587,143873534,1579449415,2020-01-19,蹲空投阴人,【吃鸡技巧】解放军教你甩狙，守株待兔，一石二鸟,2020-01-19
3,84104587,143873534,1579449404,2020-01-19,先封烟！先封烟！,【吃鸡技巧】解放军教你甩狙，守株待兔，一石二鸟,2020-01-19
4,84104587,143873534,1579449220,2020-01-19,封烟强拉,【吃鸡技巧】解放军教你甩狙，守株待兔，一石二鸟,2020-01-19
...,...,...,...,...,...,...,...
583984,328146940,192127980,1589912222,2020-05-19,你们真厉害，只是你们不知道,假如闭上双眼，你敢在黑暗中奔跑吗？,2020-05-17
583985,328146940,192127980,1589908501,2020-05-19,你们真厉害,假如闭上双眼，你敢在黑暗中奔跑吗？,2020-05-17
583986,328146940,192127980,1589908293,2020-05-19,我闭着眼走路会走歪，,假如闭上双眼，你敢在黑暗中奔跑吗？,2020-05-17
583987,328146940,192127980,1589905946,2020-05-19,国足努力了？那为什么踢不过其他国家的？,假如闭上双眼，你敢在黑暗中奔跑吗？,2020-05-17


In [258]:
# write out the data 
df_title.to_csv('共青团视频弹幕_whole.csv')

## Method without history danmaku

In [4]:
def get_danmaku(aid,df):
    # obtain cid 
    url_cid = 'https://www.bilibili.com/widget/getPageList?aid='+ str(aid)
    response_cid = requests.get(url_cid)
    json_cid = json.loads(response_cid.text)
    cid = json_cid[0]['cid']
    
    url = "https://api.bilibili.com/x/v1/dm/list.so?oid=" + str(cid)
    response = requests.get(url)
    html_doc = response.content.decode('utf-8')
    # html_doc = <d p="54.01300,1,25,16777215,1531313705,0,f1d4f711,2187083291033602">我狂铁胜率百分之85</d>
    format_doc = re.compile('<d p="(.*?)">(.*?)</d>')
    # danmu = [(('54.01300,1,25,16777215,1531313705,0,f1d4f711,2187083291033602','我狂铁胜率百分之85')),()]
    # () = tuple
    danmu = format_doc.findall(html_doc)
    
    # function of organize data
    def reformat(tuples):
        # info = '54.01300,1,25,16777215,1531313705,0,f1d4f711,2187083291033602' - str
        info = tuples[0]
        # words = '我狂铁胜率百分之85'
        words = tuples[1]
        # info = [54.01300,1,25,16777215,1531313705,0,f1d4f711,2187083291033602] - list
        # info = [弹幕出现的时间,弹幕的模式]
        info = info.split(',')
        # timestamp, readable datatime 
        #return [[['1531313705', '2018-07-11 12:55:05', '我狂铁胜率百分之85']],[],[]]
        return [info[4],str(pd.to_datetime(info[4],unit='s')),words]
    
    map_f = map(reformat, danmu)
    data=list(map_f)
    
    df_new = pd.DataFrame(data,columns=['dm_timestamp', 'dm_date', 'dm_content'])
    df_new['aid'] = str(aid)
    df_new['cid'] = str(cid)
    df_new = df_new[['aid','cid','dm_timestamp', 'dm_date', 'dm_content']]

    df = pd.concat([df, df_new], axis=0, sort=False)
    return df_new

#df_new = get_danmaku('455082139',df)
#df_new.head(2)

54.01300,1,25,16777215,1531313705,0,f1d4f711,2187083291033602  
54.01300    : 弹幕出现的时间: 以秒为单位  
1           : 弹幕的模式: 1～3 滚动弹幕 4 底端弹幕 5 顶端弹幕 6 逆向弹幕 7 精准定位 8 高级弹幕  
25          : 字号: 12 非常小 16 特小 18 小 25 中 36 大 45 很大 64 特别大  
16777215    : 字体的颜色：将 HTML 六位十六进制颜色转为十进制表示，例如 #FFFFFF 会被存储为 16777215
1531313705  :时间戳: 以毫秒为单位，基准时间为 1970-1-1 08:00:00  
0           :弹幕池: 0 普通池 1 字幕池 2 特殊池（注：目前特殊池为高级弹幕专用）  
f1d4f711    :发送者的 ID: 用于「屏蔽此弹幕的发送者」功能  
2187083291033602 : 弹幕在弹幕数据库中 rowID，用于「历史弹幕」功能  

In [5]:
df = pd.DataFrame(columns=['dm_timestamp', 'dm_date', 'dm_content'])

# go through all the videos for scraping the danmaku
for i in list(aid_list_df.aid):
    df_new = get_danmaku(i,df)
    df = pd.concat([df_new,df], axis=0, sort=False)
df.head()

,aid,cid,dm_timestamp,dm_date,dm_content
0,540647232,192597418,1589844624,2020-05-18 23:30:24,视频够客气了
1,540647232,192597418,1589844668,2020-05-18 23:31:08,哈哈哈
2,540647232,192597418,1589844691,2020-05-18 23:31:31,笑死个人
3,540647232,192597418,1589845157,2020-05-18 23:39:17,确实在美国适得其反
4,540647232,192597418,1589845309,2020-05-18 23:41:49,嗯？？？？


In [6]:
df.shape

(232470, 5)

In [76]:
# export data
df.to_csv('共青团视频弹幕_no_his.csv')

第一，实时性的差异。弹幕是一种对于内容的实时参与反馈，相当于实时短评。而评论偏向对于内容的总结性论述，往往在完整的内容消费之后产生。


第二，参与感的差异。弹幕是一种鼓励全员参与的行为，通过参与加深用户对于平台的沉浸感。评论在参与感方面鼓励性不足。


第三，交互性的差异。弹幕引导用户沉浸在交互体验中，更能加深用户对于内容的共鸣。但是评论往往是在完成内容消费后进行反馈。


第四，沉浸式的差异。弹幕不好的地方在于容易打断了用户对于内容的连续性观赏，打断用户的沉浸式体验，甚至有人提前剧透，对于悬疑剧来说是多么毁灭性的体验，评论行为一般鼓励用户在完整消费内容之后给出答案。


第五，趣味性差异。弹幕更具趣味性，评论更具理性。


第六，内容输出的价值差异。弹幕往往是一句话发表及时性感性言语，主要通过用户之间的共鸣感提升内容消费体验。而评论往往给用户更多的思考时间产出更多有理性价值的内容输出，参考性更强。

In [12]:
df.dm_date.unique()

In [13]:
df.shape

(232470, 5)

In [16]:
df

,aid,cid,dm_timestamp,dm_date,dm_content
0,540647232,192597418,1589844624,2020-05-18 23:30:24,视频够客气了
1,540647232,192597418,1589844668,2020-05-18 23:31:08,哈哈哈
2,540647232,192597418,1589844691,2020-05-18 23:31:31,笑死个人
3,540647232,192597418,1589845157,2020-05-18 23:39:17,确实在美国适得其反
4,540647232,192597418,1589845309,2020-05-18 23:41:49,嗯？？？？
...,...,...,...,...,...
995,76394931,130684654,1590234122,2020-05-23 11:42:02,美国活该被新冠入侵
996,76394931,130684654,1590236332,2020-05-23 12:18:52,祖安引路人，欢迎加入
997,76394931,130684654,1590236854,2020-05-23 12:27:34,气死我了
998,76394931,130684654,1590236874,2020-05-23 12:27:54,都是中国人，他们怎么都这样
